In [1]:
'''
    In Chapter 1, we briefly looked at the problem of trying to predict which DataSciencester users paid for premium accounts. Here we'll revisit that problem
'''#W#We never went through this in chapter 1, maybe this is from the first edition and is a mistake??? Only thing similiar I can think of is the PhD example.

"\n    In Chapter 1, we briefly looked at the problem of trying to predict which DataSciencester users paid for premium accounts. Here we'll revisit that problem\n"

In [2]:
#Super inaccurate data but there is no data given in the text
import random

def create_data(users: int):
    a = []
    while len(a) != users:
        b = []
        salary = random.randrange(25000, 110000)
        b.append(salary)
        experience = random.uniform(0, 25)
        b.append(experience)
        paid_account = random.randint(0, 1)
        b.append(paid_account)
        a.append(b)
        continue
    return a

data = create_data(200)

In [3]:
#The Problem
'''
    We have an anonymized dataset of about 200 users, containing each user's salary, her years of experience as a data scientist, and whether she paid for a premium account. 
    As usual, our data is a list of rows [experience, salary, paid_account]
'''
xs = [[1.0] + row[:2] for row in data]
ys = [row[2] for row in data]

#An obvious first attempt is to use linear regression and find the best model:
    #paid account = β0 + β1experience + β2salary + ε

In [4]:
from matplotlib import pyplot as plt 
from scratch.working_with_data import rescale
from scratch.multiple_regression import least_squares_fit, predict
from scratch.gradient_descent import gradient_step

learning_rate = 0.001
rescaled_xs = rescale(xs)
beta = least_squares_fit(rescaled_xs, ys, learning_rate, 1000, 1)
predictions = [predict(x_i, beta) for x_i in rescaled_xs]

plt.scatter(predictions, ys)
plt.xlabel("predicted")
plt.ylabel("actual")
plt.show()
#FileNotFoundError that should be in im folder but nothing is in im folder on GH

FileNotFoundError: [Errno 2] No such file or directory: 'im/working_scatter.png'

In [1]:
#The Logistic Function

def logistic(x: float) -> float:
    return 1.0 / (1 + math.exp(-x))

'''
    As its input gets large and positive, it gets closer and closer to 1. As its input gets large and negative, it gets closer and closer to 0. Additionally, it has the convenient property that its derivative is given by:  
'''

def logistic_prime(x: float) -> float:
    y = logistic(x)
    return y * (1 - y)

'''
    which we'll make use of in a bit. We'll use this to fit a model:
        yi = f(xiβ) + εi
    
    where f is the logistic function 
'''

import math 
from scratch.linear_algebra import Vector, dot

def _negative_log_likelihood(x: Vector, y: float, beta: Vector) -> float:
    """The negative log likelihood for one data point"""
    if y == 1:
        return -math.log(logistic(dot(x, beta)))
    else:
        return -math.log(1 - logistic(dot(x, beta)))

In [4]:
from typing import List

def negative_log_likelihood(xs: List[Vector],
                            ys: List[float],
                            beta: Vector) -> float:
    return sum(_negative_log_likelihood(x, y, beta) for x, y, in zip(xs, ys))

from scratch.linear_algebra import vector_sum

def _negative_log_partial_j(x: Vector, y: float, beta: Vector, j: int) -> float:
    """ The j'th partial derivative fro one data point. Here i is the index of the data point.
    """
    return -(y - logistic(dot(x,beta))) * x[j]

def _negative_log_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    """
        The gradient for one data point
    """
    return [-_negative_log_partial_j(x, y, beta) for j in range (len(beta))]

def negative_log_gradient(xs: List[Vector],
                            ys: List[float],
                            beta: Vector) -> Vector:
    return vector_sum([_negative_log_gradient(x, y, beta) for x, y in zip(xs, ys)])

In [12]:
#Applying the Model

from scratch.machine_learning import train_test_split
import random
import tqdm

random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(rescaled_xs, ys, 0.33)

learning_rate = 0.01

#pick a random starting point
beta = [random.random() for _ in range(3)]

with tqdm.trange(5000) as t:
    for epoch in t:
        gradient = negative_log_gradient(x_train, y_train, beta)
        beta = gradient_step(beta, gradient, -learning_rate)
        loss = negative_log_likelihood(x_train, y_train, beta)
        t.set_description(f"loss: {loss:.3} beta: {beta}")

#Doesn't work because of cell 4

NameError: name 'rescaled_xs' is not defined

In [13]:
from scratch.working_with_data import scale

means, stdevs =scale(xs)
beta_inscaled = [(beta[0] - beta[1] * means[1] / stdevs[1] - beta[2] * means[2] / stdevs[2]), 
                beta[1] / stdevs[1], 
                beta[2] / stdevs[2]]

FileNotFoundError: [Errno 2] No such file or directory: 'im/working_scatter.png'

In [5]:
#Goodness of Fit

true_positives = false_positives = true_negatives = false_negatives = 0

for x_i, y_i, in zip(x_test, y_test):
    prediction = logistic(dot(beta, x_i))

    if y_i == 1 and prediction >= 0.5:  #TP: paid and we predict paid
        true_positives += 1
    elif y_i == 1:                      #FN: paid and we predict unpaid
        false_negatives += 1
    elif predition >= 0.5:              #FP: unpaid and we predict paid
        false_positives     
    else:
        true_negatives += 1

precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)

'''
    This gives a precision of 75% ("when we predict paid account we're right 75% of the time") and a recall of 80% ("when a user has a paid account we predict paid accounts 80% of the time"), which is not terrible considering how little data we have
'''

#We can also pplot the predictions versus the actuals, which also shows that the model performs well:

predictions = [logistic(dot(beta, x_i) for x_i in x_test)]
plt.scatter(predictions, y_test, marker='+')
plt.xlabel("predicted probability")
plt.ylabel("actual outcome")
plt.title("Logistic Regression Predicted vs Actual")
plt.show()

NameError: name 'x_test' is not defined

In [ ]:
#Support Vector Machines

'''
    The set of points where dot(beta, x__i) equals 0 is the boundary between our classes. 
    This boundary is a hyperplane that splits the parameter space into two half-spaces corresponding to predict paid and predict unpaid.
    An alternative approach to classification is to just look for the hyperplace that "best" separates the classes in the training data. This is the idea behind the support vector machine, which finds the hyperplane that maximizes the distance to the nearest point in each class. 
    A different problem is that a separating hyperplane might not exist at all. In our "who pays?" dataset there simply is no line that perfectly separates the paid users from the unpaid users. 
    We can sometimes get around this by transforming the data into a higher dimensional space.
    If we map this dataset to two dimensions by sending the point x to (x, x**2). Suddenly it's possible to find a hyperplane that splits the data. 
    This is usually called the kernal trick because rather than actually mapping the points intop higher-dimensional space, we can use a "kernal" function to compute dot products in the higher-dimensional space and use those to find a hyperplace. 
    It's hard (and probably not a good idea) to use support vector machine without relying on specialized optimization software written by people with the appropriate expertise.
'''